In [1]:
import pandas as pd
import numpy as np
import panel as pn
pn.extension('tabulator')
import ipywidgets as widgets

import hvplot.pandas


In [10]:
# cache data to improve dashboard performance
if 'data' not in pn.state.cache.keys():

    df = pd.read_csv('C:/Users/ash/Downloads/olympics_dataset_up.csv') 
    df.columns = map(str.lower, df.columns)
    df['continent'] = df['continent'].fillna('Other')
    df['year'] = df['year'].astype(int)
    pn.state.cache['data'] = df.copy()

else: 

    df = pn.state.cache['data']
    

# Minor Data Preprocessing

In [11]:
# Fill NAs with 0s and create GDP per capita column
df = df.fillna(0)


In [12]:
# Make DataFrame Pipeline Interactive
idf = df.interactive()




# Visual 1: Olympic Medals Over Time by Continent

In [13]:
# Define Panel widgets
year_slider = pn.widgets.IntSlider(name='Year Slider', start=1895, end=2025, step=1, value=1895)
year_slider

IntSlider(end=2025, name='Year Slider', start=1895, value=1895)

In [14]:
# Radio buttons for CO2 measures
yaxis_medal_cnt = pn.widgets.RadioButtonGroup(
    name='Y axis', 
    options=['count_medals', 'cnt_gold'],
    button_type='success'
)


In [15]:
continents = ['Other', 'Asia', 'Oceania', 'Europe', 'Africa', 'North America', 'South America', 'Antarctica']

medal_pipeline = (
    idf[
        (idf.year <= year_slider) &
        (idf.continent.isin(continents))
    ]
    .groupby(['continent', 'year'])[yaxis_medal_cnt].sum()
    .to_frame()
    .reset_index()
    .sort_values(by='year')  
    .reset_index(drop=True)
)


In [16]:
medal_pipeline

In [17]:
medal_plot = medal_pipeline.hvplot(x = 'year', by='continent', y=yaxis_medal_cnt,line_width=2, title="Medal Count by Continent")
medal_plot


# Visual 2: Medals over time by continent

In [21]:
continents = ['Other', 'Asia', 'Oceania', 'Europe', 'Africa', 'North America', 'South America', 'Antarctica']
medal_pipeline_yr = (
    idf[
        (idf.year == year_slider) &
        (idf.continent.isin(continents))
    ]
    .groupby(['continent', 'year'])[yaxis_medal_cnt].sum()
    .to_frame()
    .reset_index()
    .sort_values(by='year')  
    .reset_index(drop=True)
)


In [24]:
medal_table = medal_pipeline_yr.pipe(pn.widgets.Tabulator, pagination='remote', page_size = 10, sizing_mode='stretch_width', index=No ) 

medal_table

# Visual 3: Gold vs All Scatterplot

In [25]:
continent_level_agg = df.groupby(['continent', 'year'], as_index=False)['count_medals'].sum()

continent_level_agg

idf_agg = continent_level_agg.interactive()

In [26]:
continents = ['Other', 'Asia', 'Oceania', 'Europe', 'Africa', 'North America', 'South America', 'Antarctica']


medal_scatterplot_pipeline = (
    idf_agg[
        (idf_agg.year == year_slider) &
          (idf_agg.continent.isin(continents))
       
    ]
    .groupby(['continent', 'year'])['count_medals'].sum()
    .to_frame()
    .reset_index()
    .sort_values(by='year')  
    .reset_index(drop=True)
)


In [27]:
medal_scatterplot = medal_scatterplot_pipeline.hvplot(x='count_medals', 
                                                                y='continent', 
                                                                by='continent', 
                                                                size=80, 
                                                                kind="scatter", 
                                                                alpha=0.7,
                                                                legend=False, 
                                                                height=500, 
                                                                width=500)
medal_scatterplot

# Visual 4: Bar Chart with Medals by Continent 

In [28]:
yaxis_medal_source = pn.widgets.RadioButtonGroup(
    name='Y axis', 
    options=['cnt_gold', 'cnt_silver', 'cnt_bronze'], 
    button_type='success'
)

continents_excl_world = ['Asia', 'Oceania', 'Europe', 'Africa', 'North America', 'South America', 'Antarctica']

medal_source_bar_pipeline = (
    idf[
        (idf.year == year_slider) &
        (idf.continent.isin(continents_excl_world))
    ]
    .groupby(['year', 'continent'])[yaxis_medal_source].sum()
    .to_frame()
    .reset_index()
    .sort_values(by='year')  
    .reset_index(drop=True)
)

medal_source_bar_pipeline

In [29]:
medal_source_bar_plot = medal_source_bar_pipeline.hvplot(kind='bar', 
                                                     x='continent', 
                                                     y=yaxis_medal_source, 
                                                     title='Medals by continent')
medal_source_bar_plot

# Creating Dashboard

In [ ]:
#Layout using Template
template = pn.template.FastListTemplate(
    title='Olympics Medals Dashboard', 
    sidebar=[pn.pane.Markdown("# Medals by Continents"), 
             pn.pane.Markdown("#### This data dashboard provides a comprehensive overview of the Olympic medals won by each continent throughout the history of the Games. It visually tracks the distribution and trends of gold, silver, and bronze medals over the years, allowing users to explore the performance and growth of different continents in the global sporting arena. The interactive features enable a summarized analysis of specific time periods, medal counts, and countries within each continent, offering insights into historical achievements and competitive dynamics."), 
             pn.pane.Markdown("###### Continent:'Other' includes players not representing a country and/or countries that are no longer current countries e.g. Austro-Hungarian Empire."),
             pn.pane.PNG('C:/Users/ash/Documents/projects/olympics_dashboard/medal.png', sizing_mode='scale_both'),
             pn.pane.Markdown("## Settings"),   
             year_slider],
    main=[pn.Row(pn.Column(yaxis_medal_cnt, 
                           medal_plot.panel(width=700), margin=(0,25)), 
                           medal_table.panel(width=500)), 
          pn.Row(pn.Column(medal_scatterplot.panel(width=600), margin=(0,25)), 
                 pn.Column(yaxis_medal_source, medal_source_bar_plot.panel(width=600)))],
    accent_base_color="#008000",
    header_background="#008000",
)
template.show()
template.servable()

#pn.serve(template, port=80, websocket_origin=['*'])

#template.save('Olympics_Dashboard')
;